In [1]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# Import the 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

train=pd.read_csv(r'E:\Datasets\train_numbers.csv')
train.tail()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41999,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# save the labels to a Pandas series target
target = train['label']
# Drop the label feature
X= train.drop("label",axis=1)

target = target[:2000]
X = X[:2000]

In [3]:
X.shape

(2000, 784)

<h3>Standerdize the data</h3>

In [4]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

<h3>Covariance matrix</h3>

In [5]:
import numpy as np
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

Covariance matrix 
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


<h3>Next, we perform an eigendecomposition on the covariance matrix:</h3>

In [6]:
cov_mat = np.cov(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 ..., 
 [ 0.+0.j  0.+0.j  0.+0.j ...,  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  1.+0.j]]

Eigenvalues 
[  4.08503648e+01 +0.00000000e+00j   2.93130234e+01 +0.00000000e+00j
   2.84660302e+01 +0.00000000e+00j   2.10139282e+01 +0.00000000e+00j
   1.79835900e+01 +0.00000000e+00j   1.63976364e+01 +0.00000000e+00j
   1.35489252e+01 +0.00000000e+00j   1.26351424e+01 +0.00000000e+00j
   1.15897737e+01 +0.00000000e+00j   1.05099667e+01 +0.00000000e+00j
   9.83176547e+00 +0.00000000e+00j   8.89080759e+00 +0.00000000e+00j
   8.40498313e+00 +0.00000000e+00j   8.00290182e+00 +0.00000000e+00j
   7.88908673e+00 +0.00000000e+00j   7.57913141e+00 +0.00000000e+00j
   7.08016605e+00 +0.00000000e+00j   6.84690070e+00 +0.00000000e+00j
   6.72

<h3> Selecting Principal Components</h3>
The typical goal of a PCA is to reduce the dimensionality of the original feature space by projecting it onto a smaller subspace, where the eigenvectors will form the axes. However, the eigenvectors only define the directions of the new axis, since they have all the same unit length 1.

<h6>In order to decide which eigenvector(s) can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.
In order to do so, the common approach is to rank the eigenvalues from highest to lowest in order choose the top kk eigenvectors.</h6>

In [7]:
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

print(eig_pairs)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


After sorting the eigenpairs, the next question is "how many principal components are we going to choose for our new feature subspace?" A useful measure is the so-called "explained variance," which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.
In

Here, we are reducing the 784-dimensional feature space to a 2-dimensional feature subspace, by choosing the "top 2" eigenvectors with the highest eigenvalues to construct our d×kd×k-dimensional eigenvector matrix WW

In [8]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(784,1), 
                      eig_pairs[1][1].reshape(784,1)))
print('Matrix W:\n', matrix_w)

Matrix W:
 [[ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]
 ..., 
 [ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]]


<h3>Projection Onto the New Feature Space</h3>
<p>In this last step we will use the 4×2-dimensional projection matrix W to transform our samples onto the new subspace via the equation</p>
<p>Y=X×W, where Y is a 1000×2 matrix of our transformed samples.</p>


In [9]:
Y = X_std.dot(matrix_w)

In [10]:
len(Y)

2000

In [11]:
print(Y)

[[ -5.51974243+0.j  -6.07452726+0.j]
 [ 18.86205079+0.j   5.00162345+0.j]
 [ -7.75658168+0.j  -2.25267987+0.j]
 ..., 
 [ -0.72529226+0.j   5.50369083+0.j]
 [  3.10235044+0.j  -3.66026304+0.j]
 [ -2.72342453+0.j   9.11351268+0.j]]


In [12]:
#Seperating out the columns
X1=np.real(Y[:,0])
Y1=np.real(Y[:,1])


In [13]:
# For cluster coloring in our Plotly plots, remember to also restrict the target values 
Target = target[:2000]


trace0 = go.Scatter(
    x=X1,
    y=Y1,
    mode='markers',
    name='dddd',
marker = dict(
        size = 8,
        color = Target,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )    
)

data = [trace0]
layout = go.Layout(
    title= 'Principal Component Analysis (PCA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Principal Component',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Principal Component',
        ticklen= 5,
        gridwidth= 2,
    ),showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')


<h1>PCA using inbuilt functions</h1>

In [15]:
# Call the PCA method with 5 components. 
pca = PCA(n_components=5)
pca.fit(X_std)
X_5d = pca.transform(X_std)


In [16]:
trace0 = go.Scatter(
    x = X_5d[:,0],
    y = X_5d[:,1],
    name = Target,
    hoveron = Target,
    mode = 'markers',
    text = Target,
    showlegend = False,
    marker = dict(
        size = 8,
        color = Target,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [trace0]

layout = go.Layout(
    title= 'Principal Component Analysis (PCA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Principal Component',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Principal Component',
        ticklen= 5,
        gridwidth= 2,
    ),showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')